# Quickstart: how to setup an HTS and a Reservoir database

In this notebook we setup a new database



Import of standard packages:

In [1]:
import numpy as np
import time

Import of `DAna` packages:

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
    
from dana.hts import Hts
from dana.reservoir import OrdersReservoir, create_mi, create_ci

## Historical Time Series database



We define a function to generate random time series of traded prices and volumes. A time series is a list of dictionaries. Each dictionary stores the information of a one-minute of trades, i.e. a reference price (the vwap or the close price) and the total volume traded within the minute.

In [3]:
def generate_ts(date, start, end, v_minute, volatility):
    
    ''' Generate random time series of prices and volumes '''

    # price start
    price = 100.

    # empty output
    ts = []
    for mins in range(start, end):
        record = {
            'date': date,
            'mins': mins,
            'price': price,
            'volume': v_minute
                 }
        ts.append(record)
        price += volatility * 2.*(np.random.rand()-0.5)

    return ts


We genenerate an instance of the Hts object. We connect to the `localhost` server and to the `ts_example` database. You need a MongoDB instance started.

In [4]:
hts = Hts(engine_url='mongodb://localhost:27017/', db_name='ts_example')

We consider a market composed of `n_symbols`. We consider a range of `n_dates` available dates from the Julian date 200000. The market is open between `market_open` and `market_close`. For each symbol a constant volume `v_minute_market` is traded each minute.

In [5]:
# available dates
date = 200000
n_dates = 100
date_list = list(range(date, date+n_dates))

# available symbols
n_symbols = 10
symbol_list = ['symbol_'+str(i) for i in range(n_symbols)]

# market open and close
market_open = 100
market_close = 200

# volume per minute traded in the market
v_minute_market = 100

# price volatility of the price time series for each symbol and date
volatility = 1.

For each couple of day and symbol, we generate a random time series that we store in `hts`. For each symbol, we add an index.

In [6]:
for symbol in symbol_list:

    # drop symbol
    hts.drop_symbol(symbol)

    for date in date_list:
        # generate time series
        ts = generate_ts(date, market_open, market_close, v_minute_market, volatility)

        # insert ts
        hts.insert_ts(symbol, ts)

    # index on dates
    hts.add_index(symbol)

## Order Reservoir database


We define a function to generate random orders. Each order is represented by a dictionary. The information stored in the dictionray is:
* `mgr`: the code of the order manager
* `bkr`: the code of the order broker
* `symbol`: the traded symbol
* `sign`: the order sign (-1 sell, +1 buy)
* `date`: the trade date (Julian date)
* `min_start`: start minute of the order (minutes from midnight)
* `min_end`: end minute of the order (minutes from midnight)
* `volume`: the order volume (number of units)
* `price`: the realised vwap of the order
* `n_trades`: the number of trades of the order
* `duration_ph`: the order duration in physical time (minutes)
* `ci`: cross information with the market
* `mi`: market impact information


In [7]:
def random_times(min_start, min_end):

    # random start and end time
    t_random = [0, 0]
    while t_random[0] == t_random[1]:
        t_random = np.random.randint(min_start, min_end, 2)

    t_start = int(str(min(t_random)))
    t_end = int(str(max(t_random)))
    return t_start, t_end


def create_order(symbol, date, min_start, min_end, v_minute):

    """ Generate a random order """

    # maximum number of trades per order
    n_max = 10

    # price base
    price_base = 100.

    order = {
        'mgr': 'mgr_test',
        'bkr': 'bkr_test',
        'symbol': symbol,
        'sign': 1 if np.random.rand() < 0.5 else -1,
        'date': date,
        'min_start': min_start,
        'min_end': min_end,
        'volume': v_minute * (min_end-min_start),
        'price': price_base + np.random.rand(),
        'ntrades': np.random.randint(0, n_max),
        'duration_ph': min_end-min_start,
        'ci': {'available': False},
        'mi': {'available': False}
    }

    return order


Generate an insance of a OrderReservoir. We connect to the database `orders_example`.

In [8]:
reservoir = OrdersReservoir(engine_url='mongodb://localhost:27017/', db_name='orders_example')

Parameters:

In [9]:
# number of orders for each day
n_orders_day = 20

We remove all the orders present in the reservoir.

In [10]:
print('Empty reservoir:')
t0 = time.time()
reservoir.empty_reservoir()
print(time.time() - t0)

Empty reservoir:
0.04789614677429199


Create a random sample of orders. For each order we retrieve the cross market informaiton `ci` and the market impact information `mi` by means of the `cerate_ci` and `create_mi` functions. The function `create_ci` calculates:
* `duration_vol`: the order duration in volume time
* `pr_day`: the participation rate per day
* `pr_period`: the participation rate per perio
* `impact_co`: the impact close-open
* `impact_vo`: the impact vwap-open
* `impact_vv`: the impact vwap-vwap
The function `create_mi` calculate the impact during the execution of the order price(t) vs price(0). We add `mi` and `ci` to the order and we store the information in the database.

In [12]:
for symbol in symbol_list:
    for date in date_list:
        
        # create empty order list
        order_list = []
        
        # create orders
        for i_order in range(n_orders_day):
            min_start, min_end = random_times(market_open, market_close)
            v_minute_order = np.random.randint(v_minute_market)
            order_list.append(create_order(symbol=symbol, date=date, min_start=min_start, min_end=min_end, v_minute=v_minute_order))
        
        # get the day candle
        candle_day = hts.get_candle(symbol, date, market_open, market_close)

        for order in order_list:
            
            # create the cross information and add to the order
            candle_period = hts.get_candle(symbol, date, order['min_start'], order['min_end'])
            ci = create_ci(order, candle_day, candle_period, volatility=1.)
            order['ci'] = ci

            # create the  market impact information and assign to the order
            time_series = hts.get_ts(symbol, date, order['min_start'], order['min_end'])
            mi = create_mi(order, time_series, volatility=1.)
            order['mi'] = mi
        
        # add the orders to the reservoir
        reservoir.insert_orders(order_list)